# VacationPy Analysis

In [1]:
# Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

In [2]:
# Define a path to the csv file exported in Part 1
csvpath = '../WeatherPy/Weather_Output/cities.csv'

In [3]:
# Load the csv to a Pandas DataFrame
cities_df = pd.read_csv(csvpath)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,lebu,-37.6167,-73.6500,53.29,78.0,68.0,12.77,CL,2022-04-24
1,ancud,-41.8697,-73.8203,46.22,78.0,69.0,24.34,CL,2022-04-24
2,ushuaia,-54.8000,-68.3000,44.26,65.0,75.0,5.75,AR,2022-04-24
3,fortuna,40.5982,-124.1573,61.93,75.0,0.0,11.50,US,2022-04-24
4,kapaa,22.0752,-159.3190,80.58,83.0,75.0,17.27,US,2022-04-24


## Humidity Heatmap

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

In [5]:
# Create a variable, locations, to store 'Lat' and 'Lng' 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Create a variable, humidity, to store city humidity values
humidity = cities_df['Humidity'].astype(float)

In [6]:
# Create a Heatmap using latitude and longitude for the location of each city and humidty as the weight
# Configure the map
fig = gmaps.figure(center=(5,0), zoom_level=2, layout={'width': '975px','height': '700px',
                                                        'padding': '3px','border': '1px solid black'})

# Create the heat layer and add specifications
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 2.5)

# Add the heat layer to the map
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', padding='3px', width='975px'))

## Vacation Hotels based on Ideal Weather Conditions

In [7]:
# Create a new dataframe based on desired weather criteria for vacation
hotel_df = cities_df.loc[(cities_df['Max Temp'] > 80) & (cities_df['Max Temp'] <= 90) 
                                 & (cities_df['Humidity'] <= 65) & (cities_df['Cloudiness'] == 0) 
                                 & (cities_df['Wind Speed'] <= 10)]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,cabo san lucas,22.8909,-109.9124,82.06,61.0,0.0,7.56,MX,2022-04-24
96,russell,32.3502,-85.1999,82.11,44.0,0.0,0.00,US,2022-04-24
233,tessalit,20.1986,1.0114,82.02,8.0,0.0,9.95,ML,2022-04-24
250,ashoknagar,24.5667,77.7167,85.66,14.0,0.0,8.30,IN,2022-04-24
276,gangapur,26.4833,76.7167,86.14,13.0,0.0,6.51,IN,2022-04-24
309,rancho palos verdes,33.7445,-118.3870,88.99,42.0,0.0,9.80,US,2022-04-24
346,rajapur,25.3833,81.1500,87.85,11.0,0.0,3.02,IN,2022-04-24
392,jawar,23.0000,76.5000,83.97,21.0,0.0,6.98,IN,2022-04-24
427,dostpur,26.2833,82.4667,84.43,14.0,0.0,7.58,IN,2022-04-24
512,talen,23.5667,76.7167,86.18,15.0,0.0,8.39,IN,2022-04-24


In [8]:
# Add a blank 'Hotel Name' column to the DataFrame
hotel_df['Hotel Name'] = ''
hotel_df

/Users/graceolson/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
44,cabo san lucas,22.8909,-109.9124,82.06,61.0,0.0,7.56,MX,2022-04-24,
96,russell,32.3502,-85.1999,82.11,44.0,0.0,0.00,US,2022-04-24,
233,tessalit,20.1986,1.0114,82.02,8.0,0.0,9.95,ML,2022-04-24,
250,ashoknagar,24.5667,77.7167,85.66,14.0,0.0,8.30,IN,2022-04-24,
276,gangapur,26.4833,76.7167,86.14,13.0,0.0,6.51,IN,2022-04-24,
309,rancho palos verdes,33.7445,-118.3870,88.99,42.0,0.0,9.80,US,2022-04-24,
346,rajapur,25.3833,81.1500,87.85,11.0,0.0,3.02,IN,2022-04-24,
392,jawar,23.0000,76.5000,83.97,21.0,0.0,6.98,IN,2022-04-24,
427,dostpur,26.2833,82.4667,84.43,14.0,0.0,7.58,IN,2022-04-24,
512,talen,23.5667,76.7167,86.18,15.0,0.0,8.39,IN,2022-04-24,


In [9]:
# Set a base URL for Google Places API and parameters to search for hotels within 5,0000 meters of desired cities
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
params = {'type' : 'hotel',
          'radius' : 5000,
          'key' : g_key}

# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    #Set a variable for retriving city coordinates from the DataFrame
    coords = f'{row["Lat"]},{row["Lng"]}'
    
    # Add location to parameters dictionary
    params['location'] = coords
    
    # Set a variable for retrieving the city name from the DataFrame
    city = hotel_df['City']
    
    # Assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response['results']
    
    # Find the second hotel from the results (the first hotel for a lot of my locations was the name of the city itself, so I selected the second hotel from the results for more interesting data)
    # Store results in the 'Hotel Name' column of the DataFrame
    try:
        print(f"Closest hotel in {row['City']} is {results[1]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[1]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 44: cabo san lucas.
Closest hotel in cabo san lucas is Hotel Tesoro Los Cabos.
------------
Retrieving Results for Index 96: russell.
Closest hotel in russell is SHON & MANDY.
------------
Retrieving Results for Index 233: tessalit.


/Users/graceolson/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel in tessalit is Marisquería el peruano.
------------
Retrieving Results for Index 250: ashoknagar.
Closest hotel in ashoknagar is Bank of India.
------------
Retrieving Results for Index 276: gangapur.
Closest hotel in gangapur is indian cloth store Radha Krishna Market.
------------
Retrieving Results for Index 309: rancho palos verdes.
Closest hotel in rancho palos verdes is Terranea Resort.
------------
Retrieving Results for Index 346: rajapur.
Closest hotel in rajapur is BANK OF BRODA.
------------
Retrieving Results for Index 392: jawar.
Closest hotel in jawar is Khushi sales Indian Oil Petrol Pump.
------------
Retrieving Results for Index 427: dostpur.
Closest hotel in dostpur is Bank of Baroda.
------------
Retrieving Results for Index 512: talen.
Closest hotel in talen is Pakiza.
------------
Retrieving Results for Index 549: villa rica.
Closest hotel in villa rica is Mirror Lake Golf Club.
------------


In [10]:
# Display the new DataFrame with the hotels found using the Google Places API
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
44,cabo san lucas,22.8909,-109.9124,82.06,61.0,0.0,7.56,MX,2022-04-24,Hotel Tesoro Los Cabos
96,russell,32.3502,-85.1999,82.11,44.0,0.0,0.00,US,2022-04-24,SHON & MANDY
233,tessalit,20.1986,1.0114,82.02,8.0,0.0,9.95,ML,2022-04-24,Marisquería el peruano
250,ashoknagar,24.5667,77.7167,85.66,14.0,0.0,8.30,IN,2022-04-24,Bank of India
276,gangapur,26.4833,76.7167,86.14,13.0,0.0,6.51,IN,2022-04-24,indian cloth store Radha Krishna Market
309,rancho palos verdes,33.7445,-118.3870,88.99,42.0,0.0,9.80,US,2022-04-24,Terranea Resort
346,rajapur,25.3833,81.1500,87.85,11.0,0.0,3.02,IN,2022-04-24,BANK OF BRODA
392,jawar,23.0000,76.5000,83.97,21.0,0.0,6.98,IN,2022-04-24,Khushi sales Indian Oil Petrol Pump
427,dostpur,26.2833,82.4667,84.43,14.0,0.0,7.58,IN,2022-04-24,Bank of Baroda
512,talen,23.5667,76.7167,86.18,15.0,0.0,8.39,IN,2022-04-24,Pakiza


In [11]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Plot hotel markers on top of the heatmap
# Configure the map
fig = gmaps.figure(center=(5,0), zoom_level=2, layout={'width': '975px','height': '700px',
                                                        'padding': '3px','border': '1px solid black'})

# Create the markers layer and add specifications
hotel_layer = gmaps.marker_layer(hotel_locations, info_box_content = hotel_info)

# Add the heat and hotel layers to the map and display the figure
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', padding='3px', width='975px'))